## IBM Data Science Capstone Week #3 pt. 2

#### Import/install necessary libraries

In [68]:
import pandas as pd
import numpy as np

! pip install geocoder

import geocoder
print('\ngecoder library installed and imported')


gecoder library installed and imported


### I am using the code from Part 1 of this project to reproduce our dataset of interest

In [12]:
can_postal_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_can_postal = pd.read_html(can_postal_link)[0]
# Use boolean mask to remove unassigned Borough's
df_can_postal = df_can_postal[df_can_postal.Borough != 'Not assigned']
# Group dataframe and aggregate neighborhood into comma-separated list
df_can_postal = (df_can_postal.groupby(['Postal Code', 'Borough'])
    .apply(lambda grp: ', '.join(grp.Neighbourhood.tolist()))
    .to_frame()
    .rename({0:'Neighbourhood'}, axis=1)
    .reset_index())
# Replace unassigned neighborhoods with borough names
df_can_postal['Neighbourhood'] = df_can_postal.apply(lambda x: x.Borough 
                                                     if x.Neighbourhood == 'Not assigned' 
                                                     else x.Neighbourhood, axis=1)

df_can_postal.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


### Google geocoder API did not work for me. I discovered I could use the ArcGIS library instead.

def get_lat_long(postal_code):
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    return latitude, longitude

lat_lng = df_can_postal['Postal Code'].apply(get_lat_long)

df_can_postal['Latitude'] = lat_lng.apply(lambda x: x[0])
df_can_postal['Longitude'] = lat_lng.apply(lambda x: x[1])

### Here is my result set.
#### The lat-long coordinates returned by the ArcGIS API are slightly and insignificantly different from the results Coursera has given using the Google API.

In [62]:
pd.set_option('display.max_rows', 500)

df_can_postal

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.72582,-79.26461
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.71289,-79.28506
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.69510,-79.26466
